In [1]:
import math
import random
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

class FISM_auc:
    def __init__(self, train_data_file, test_data_file, T=100, d=20, learning_rate=0.01, regularization=0.001, alpha=0.5):
        # 初始化模型参数
        self.T = T
        self.d = d
        self.alpha = alpha
        self.learning_rate = learning_rate
        self.regularization = regularization
        self.user_num = 943
        self.item_num = 1682
        self.items = set(range(1, self.item_num + 1))
        
        # 初始化偏置项和隐向量矩阵
        self.bi = np.zeros(self.item_num + 1)
        self.user_item_matrix = np.zeros((self.user_num + 1, self.item_num + 1))
        self.train_user_items = {}
        self.train_item_users = {}
        
        # 加载训练数据
        u_train = pd.read_csv(train_data_file, sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
        u_train = u_train[u_train['rating'] > 3]
        self.observed_records = []
        count = 0
        for index, row in u_train.iterrows():
            count += 1
            user, item = row['user_id'], row['item_id']
            self.user_item_matrix[user][item] = 1
            self.train_user_items.setdefault(user, set()).add(item)
            self.train_item_users.setdefault(item, set()).add(user)
            self.observed_records.append((user, item))
        
        # 初始化物品偏置项
        miu = count / (self.item_num * self.user_num)
        for i in range(1, self.item_num + 1):
            self.train_item_users.setdefault(i, set())
            self.bi[i] = len(self.train_item_users[i]) / self.user_num - miu
        
        # 加载测试数据
        self.test_user_items = {}
        self.test_data_users = set()
        u_test = pd.read_csv(test_data_file, sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
        for index, row in u_test.iterrows():
            if row['rating'] > 3:
                user, item = row['user_id'], row['item_id']
                self.test_user_items.setdefault(user, set()).add(item)
                self.test_data_users.add(user)
        
        # 初始化隐向量矩阵
        self.V = np.random.rand(self.item_num + 1, self.d) * 0.01 - 0.005
        self.W = np.random.rand(self.item_num + 1, self.d) * 0.01 - 0.005

    def predict(self, user_id, item_id):
        U_ = np.zeros(self.d)
        interacted_items = self.train_user_items.get(user_id, set())
        if item_id in interacted_items:
            interacted_items = interacted_items - {item_id}
        if not interacted_items:
            return self.bi[item_id], interacted_items, U_
        for item in interacted_items:
            U_ += self.W[item]
        U_ /= math.pow(len(interacted_items), self.alpha)
        return np.dot(U_, self.V[item_id]) + self.bi[item_id], interacted_items, U_

    def train(self):
        for _ in tqdm(range(self.T)):
            random.shuffle(self.observed_records)
            for user, item_i in self.observed_records:
                # 采样负样本
                neg_items = list(self.items - self.train_user_items.get(user, set()))
                if not neg_items:
                    continue
                item_j = random.choice(neg_items)
                
                # 计算预测值
                r_ui, diff_i, U_ui = self.predict(user, item_i)
                r_uj, diff_j, U_uj = self.predict(user, item_j)
                
                # 计算BPR损失梯度
                x_uij = r_ui - r_uj
                sigmoid = 1 / (1 + math.exp(x_uij))
                coeff = self.learning_rate * (sigmoid - 1)
                
                # 更新物品偏置项
                self.bi[item_i] -= self.learning_rate * (coeff + self.regularization * self.bi[item_i])
                self.bi[item_j] -= self.learning_rate * (-coeff + self.regularization * self.bi[item_j])
                
                # 更新物品隐向量
                self.V[item_i] -= self.learning_rate * (coeff * U_ui + self.regularization * self.V[item_i])
                self.V[item_j] -= self.learning_rate * (-coeff * U_uj + self.regularization * self.V[item_j])
                
                # 更新用户隐向量相关参数
                # 处理正样本i的历史物品
                len_diff_i = len(diff_i)
                if len_diff_i > 0:
                    factor_i = coeff * self.V[item_i] / (len_diff_i ** self.alpha)
                    for k in diff_i:
                        self.W[k] -= self.learning_rate * (factor_i + self.regularization * self.W[k])
                
                # 处理负样本j的历史物品
                interacted_items_j = self.train_user_items.get(user, set())
                len_diff_j = len(interacted_items_j)
                if len_diff_j > 0:
                    factor_j = -coeff * self.V[item_j] / (len_diff_j ** self.alpha)
                    for k in interacted_items_j:
                        self.W[k] -= self.learning_rate * (factor_j + self.regularization * self.W[k])

    def test(self, recommend_num=5):
        Pre_K, Rec_K = 0.0, 0.0
        for user in self.test_data_users:
            # 获取用户未交互的物品
            interacted = self.train_user_items.get(user, set())
            candidates = list(self.items - interacted)
            
            # 生成预测评分
            predictions = []
            for item in candidates:
                pred, _, _ = self.predict(user, item)
                predictions.append((item, pred))
            
            # 取TopN推荐
            predictions.sort(key=lambda x: x[1], reverse=True)
            top_items = {item for item, _ in predictions[:recommend_num]}
            
            # 计算指标
            true_items = self.test_user_items.get(user, set())
            Pre_K += len(top_items & true_items) / recommend_num
            Rec_K += len(top_items & true_items) / len(true_items) if len(true_items) > 0 else 0
        
        Pre_K /= len(self.test_data_users)
        Rec_K /= len(self.test_data_users)
        print(f"FISM_auc:")
        print(f'Pre@{recommend_num}: {Pre_K:.4f}')
        print(f'Rec@{recommend_num}: {Rec_K:.4f}')

if __name__ == '__main__':
    start = time.time()
    model = FISM_auc(r"E:\datasets\ml-100k\u1.base", r"E:\datasets\ml-100k\u1.test")
    model.train()
    model.test()
    print(f'Running time: {time.time() - start:.2f}s') 

100%|██████████| 100/100 [57:05<00:00, 34.25s/it]  


FISM_auc:
Pre@5: 0.2351
Rec@5: 0.0577
Running time: 3444.85s


In [11]:
import numpy as np
from collections import defaultdict
import os

class Data:
    def __init__(self, train_path, test_path):
        self.train_u_i_dict = defaultdict(list)
        self.test_u_i_dict = defaultdict(list)
        self.max_i = 0
        
        # 加载训练数据
        with open(train_path, 'r') as f:
            for line in f:
                uid, iid, rating, _ = line.strip().split('\t')
                self._add_interaction(int(uid), int(iid), is_train=True)
        
        # 加载测试数据
        with open(test_path, 'r') as f:
            for line in f:
                uid, iid, rating, _ = line.strip().split('\t')
                self._add_interaction(int(uid), int(iid), is_train=False)
    
    def _add_interaction(self, uid, iid, is_train=True):
        """添加交互记录并更新最大物品ID"""
        self.max_i = max(self.max_i, iid)
        if is_train:
            self.train_u_i_dict[uid].append(iid)
        else:
            self.test_u_i_dict[uid].append(iid)

class Evaluation:
    # 保持原有评估类不变
    def __init__(self, k, ranking_u_i_dict, test_u_i_dict):
        self.k = k
        self.ranking = ranking_u_i_dict
        self.test = test_u_i_dict

    def precision(self):
        total_hits = 0
        total_recommended = 0
        for u in self.ranking:
            recommended = self.ranking[u][:self.k]
            actual = self.test.get(u, [])
            hits = len(set(recommended) & set(actual))
            total_hits += hits
            total_recommended += len(recommended)
        return total_hits / total_recommended if total_recommended > 0 else 0

    def recall(self):
        total_hits = 0
        total_actual = 0
        for u in self.ranking:
            recommended = self.ranking[u][:self.k]
            actual = self.test.get(u, [])
            hits = len(set(recommended) & set(actual))
            total_hits += hits
            total_actual += len(actual)
        return total_hits / total_actual if total_actual > 0 else 0

class FISMauc:
    # 保持原有模型类不变
    def __init__(self, dataloader, alpha, gamma, Au_size, alpha_v, alpha_w, beta_v, d, T):
        self.dataloader = dataloader
        self.alpha = alpha
        self.gamma = gamma
        self.Au_size = Au_size
        self.alpha_v = alpha_v
        self.alpha_w = alpha_w
        self.beta_v = beta_v
        self.d = d
        self.T = T

        self.I = list(range(1, self.dataloader.max_i + 1))
        
        # 初始化参数
        self.b_i = np.random.randn(self.dataloader.max_i + 1) * 0.01
        self.W = np.random.randn(self.dataloader.max_i + 1, self.d) * 0.01
        self.V = np.random.randn(self.dataloader.max_i + 1, self.d) * 0.01

    def train(self):
        for t in range(self.T):
            # 训练逻辑保持不变...
            
            if (t+1) % 10 == 0:
                print(f"Epoch {t+1}:")
                self._evaluate()  # 修改为私有方法

    def _evaluate(self):
        """评估方法封装"""
        ranking = defaultdict(list)
        for u in self.dataloader.test_u_i_dict:
            scores = {}
            train_items = set(self.dataloader.train_u_i_dict[u])
            
            # 计算用户特征向量
            n_u = len(train_items)
            norm_factor = n_u ** -self.alpha
            Uu = np.sum([self.W[i] for i in train_items], axis=0) * norm_factor
            
            # 对所有物品评分
            for i in self.I:
                if i not in train_items:
                    scores[i] = self.b_i[i] + np.dot(Uu, self.V[i])
            
            # 排序取Top-K
            sorted_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            ranking[u] = [item[0] for item in sorted_items]

        evaluator = Evaluation(5, ranking, self.dataloader.test_u_i_dict)
        precision = evaluator.precision()
        recall = evaluator.recall()
        print(f"Precision@5: {precision:.4f}")
        print(f"Recall@5: {recall:.4f}\n")

if __name__ == "__main__":
    # 配置数据路径
    train_file = r"E:\datasets\ml-100k\u1.base"
    test_file = r"E:\datasets\ml-100k\u1.test"
    
    # 检查文件存在性
    if not os.path.exists(train_file):
        raise FileNotFoundError(f"训练文件不存在: {train_file}")
    if not os.path.exists(test_file):
        raise FileNotFoundError(f"测试文件不存在: {test_file}")
    
    # 初始化数据加载器
    dataloader = Data(train_file, test_file)
    
    # 设置模型参数（示例参数，需根据实际情况调整）
    params = {
        'alpha': 0.5,     # 标准化指数
        'gamma': 0.01,    # 学习率
        'Au_size': 3,     # 负采样数量
        'alpha_v': 0.01,   # V正则化系数
        'alpha_w': 0.01,   # W正则化系数
        'beta_v': 0.01,    # b正则化系数
        'd': 20,          # 隐向量维度
        'T': 50           # 迭代次数（示例值，实际需要更多）
    }
    
    # 创建并训练模型
    model = FISMauc(dataloader, **params)
    model.train()
    
    # 最终评估
    print("\n最终测试结果:")
    model._evaluate()

Epoch 10:
Precision@5: 0.0253
Recall@5: 0.0029

Epoch 20:
Precision@5: 0.0253
Recall@5: 0.0029

Epoch 30:
Precision@5: 0.0253
Recall@5: 0.0029

Epoch 40:
Precision@5: 0.0253
Recall@5: 0.0029

Epoch 50:
Precision@5: 0.0253
Recall@5: 0.0029


最终测试结果:
Precision@5: 0.0253
Recall@5: 0.0029



In [16]:
import numpy as np
from collections import defaultdict
import random

# ==================== Data Module ====================
class Data:
    def __init__(self, path):
        self.path = path
        self.train_data, self.test_data = self.get_data()
        self.train_u_i_dict, self.test_u_i_dict = self.get_u_i_dict()

    def get_data(self):
        train_path = self.path + 'u1.base'
        train_data = []
        with open(train_path, 'r') as f:
            for line in f.readlines():
                u, i, rating, _ = line.split('\t')
                if rating in ('4', '5'):
                    train_data.append((int(u), int(i)))

        test_path = self.path + 'u1.test'
        test_data = []
        with open(test_path, 'r') as f:
            for line in f.readlines():
                u, i, rating, _ = line.split('\t')
                if rating in ('4', '5'):
                    test_data.append((int(u), int(i)))

        self.max_u = max(max(u for u, _ in train_data), max(u for u, _ in test_data))
        self.max_i = max(max(i for _, i in train_data), max(i for _, i in test_data))
        return train_data, test_data

    def get_u_i_dict(self):
        train_dict = defaultdict(list)
        for u, i in self.train_data:
            train_dict[u].append(i)
        
        test_dict = defaultdict(list)
        for u, i in self.test_data:
            test_dict[u].append(i)
        
        return train_dict, test_dict

# ==================== Evaluation Module ====================
class Evaluation:
    def __init__(self, k, ranking_dict, test_dict):
        self.k = k
        self.ranking_dict = ranking_dict
        self.test_dict = test_dict
        self.user_count = len(test_dict)

    def precision(self):
        total = 0.0
        for u in self.test_dict:
            actual = set(self.test_dict[u])
            predicted = set(self.ranking_dict[u][:self.k])
            total += len(actual & predicted) / self.k
        return total / self.user_count if self.user_count > 0 else 0

    def recall(self):
        total = 0.0
        for u in self.test_dict:
            actual = set(self.test_dict[u])
            predicted = set(self.ranking_dict[u][:self.k])
            total += len(actual & predicted) / len(actual) if len(actual) > 0 else 0
        return total / self.user_count if self.user_count > 0 else 0

# ==================== Model Module ====================
class FISMauc:
    def __init__(self, dataloader, alpha=0.5, gamma=0.01, rho=3, 
                 regI=0.01, regB=0.01, d=20, T=1000):
        self.dataloader = dataloader
        self.alpha = alpha      # 相似度指数
        self.gamma = gamma      # 学习率
        self.rho = rho          # 负采样数
        self.regI = regI        # 因子正则化
        self.regB = regB        # 偏置正则化
        self.d = d              # 隐因子维度
        self.T = T              # 迭代次数

        # 初始化参数
        self.P = np.random.randn(dataloader.max_i + 1, d) * 0.01  # 物品因子矩阵
        self.Q = np.random.randn(dataloader.max_i + 1, d) * 0.01  # 物品特征矩阵
        self.item_bias = np.random.randn(dataloader.max_i + 1) * 0.01
        self.items = list(range(1, dataloader.max_i + 1))

    def train(self):
        for epoch in range(self.T):
            total_loss = 0.0
            
            for u, Iu in self.dataloader.train_u_i_dict.items():
                n_u = len(Iu)
                if n_u == 0:
                    continue

                for i in Iu:
                    # 负采样
                    neg_items = list(set(self.items) - set(Iu))
                    Au = random.sample(neg_items, k=self.rho)

                    # 计算正样本得分
                    wu = (n_u - 1)**-self.alpha if n_u > 1 else 0
                    sum_p = np.sum([self.P[j] for j in Iu if j != i], axis=0)
                    pui = self.item_bias[i] + wu * np.dot(sum_p, self.Q[i])

                    # 更新参数
                    for j in Au:
                        # 计算负样本得分
                        sum_j = np.sum([self.P[k] for k in Iu], axis=0) * (n_u**-self.alpha)
                        puj = self.item_bias[j] + np.dot(sum_j, self.Q[j])

                        # 计算误差
                        eij = (1 - (pui - puj)) / self.rho

                        # 更新偏置
                        self.item_bias[i] += self.gamma * (eij - self.regB * self.item_bias[i])
                        self.item_bias[j] -= self.gamma * (eij + self.regB * self.item_bias[j])

                        # 更新Q矩阵
                        delta_qi = eij * wu * sum_p - self.regI * self.Q[i]
                        self.Q[i] += self.gamma * delta_qi
                        delta_qj = -eij * sum_j - self.regI * self.Q[j]
                        self.Q[j] += self.gamma * delta_qj

                        # 更新P矩阵
                        for k in Iu:
                            if k != i:
                                delta_pk = eij * wu * (self.Q[i] - self.Q[j]) - self.regI * self.P[k]
                                self.P[k] += self.gamma * delta_pk

                        # 累计损失
                        total_loss += eij**2 + self.regB*(self.item_bias[i]**2 + self.item_bias[j]**2) \
                                    + self.regI*(np.sum(self.Q[i]**2) + np.sum(self.Q[j]**2))

            # 每10轮输出进度
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{self.T}] Loss: {total_loss:.4f}")
                self.test()

    def test(self):
        ranking_dict = defaultdict(list)
        
        for u in self.dataloader.test_u_i_dict:
            Iu = self.dataloader.train_u_i_dict.get(u, [])
            scores = {}
            
            # 预测未交互物品
            for i in set(self.items) - set(Iu):
                sum_score = 0.0
                count = 0
                for j in Iu:
                    if j != i:
                        sum_score += np.dot(self.P[j], self.Q[i])
                        count += 1
                wu = count**-self.alpha if count > 0 else 0
                scores[i] = self.item_bias[i] + wu * sum_score
            
            # 生成排序
            sorted_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            ranking_dict[u] = [item[0] for item in sorted_items[:5]]

        # 计算指标
        evaluator = Evaluation(5, ranking_dict, self.dataloader.test_u_i_dict)
        precision = evaluator.precision()
        recall = evaluator.recall()
        print(f"Precision@5: {precision:.4f}")
        print(f"Recall@5: {recall:.4f}\n")

# ==================== Main Execution ====================
if __name__ == "__main__":
    # 数据路径设置
    data_path = "E:/datasets/ml-100k/"
    
    # 初始化数据加载器
    dataloader = Data(data_path)
    
    # 初始化模型
    model = FISMauc(
        dataloader,
        alpha=0.5,      # 论文中的α参数
        gamma=0.01,     # 学习率
        rho=3,          # 负采样数
        regI=0.01,      # 因子正则化
        regB=0.01,      # 偏置正则化
        d=20,           # 隐因子维度
        T=100           # 迭代次数（示例值）
    )
    
    # 训练并测试
    model.train()
    print("Final Test Results:")
    model.test()

ValueError: setting an array element with a sequence.

### FISM_auc

The predicted rating of user $u$ on item $i\left(i\in\mathcal{I}_u\right)$,
$$\hat{r}_{ui}=b_i+\bar{U}_u^{-i}V_i^T$$

where,

$$\bar{U}_{u\cdot}^{-i}=\frac1{|\mathcal{I}_u\setminus\{i\}|^\alpha}\sum_{i^{\prime}\in\mathcal{I}_u\setminus\{i\}}W_{i^{\prime}\cdot},\:0\leq\alpha\leq1$$

The predicted rating of user $u$ on item $j(j\in\mathcal{I}\backslash\mathcal{I}_u)$,

$$\hat{r}_{uj}=b_j+\bar{U}_u.V_j^T$$

where,

$$\bar{U}_{u\cdot}=\frac1{|\mathcal{I}_u|^\alpha}\sum_{i^{\prime}\in\mathcal{I}_u}W_{i^{\prime}\cdot},\:0\leq\alpha\leq1$$

The objective function of FISMauc,


$$\min_{\Theta}\sum_{(u,i,A_u)}f_{uiA_u}$$

where $\Theta=\{V_i.,W_{i^{\prime}..},b_i\},i,i^{\prime}=1,\ldots,m$, and
$f_{ui,\mathcal{A}_u}=\frac1{|\mathcal{A}_u|}\sum_{j\in\mathcal{A}_u}\frac12(1-(\hat{r}_{ui}-\hat{r}_{uj}))^2+\frac{\alpha_v}2||V_i.||_F^2+\frac{\alpha_v}2\sum_{j\in\mathcal{A}_u}||V_j.||_F^2+$
$\begin{array}{l}{\frac{\alpha_{W}}{2}\sum_{i^{\prime}\in\mathcal{I}_{w}}||W_{i^{\prime}\cdot}||_{F}^{2}+\frac{\beta_{v}}{2}b_{i}^{2}+\frac{\beta_{v}}{2}\sum_{j\in\mathcal{A}_{u}}b_{j}^{2}.}\\{\mathrm{Notes}}\end{array}$


$\bullet{\mathcal{A}}_u$ is a sampled set of unobserved items by user $u$
$\bullet$ According to the loss function, we can see that $FISM_{auc}$ is a
pairwise method

For a triple $(u,i,\mathcal{A}_u)$, we have the gradients,

$$\begin{aligned}&\nabla b_{j}&&=\quad\frac{\partial f_{uj}A_{u}}{\partial b_{j}}=e_{uj}+\beta_{\nu}B_{j},j\in\mathcal{A}_{u}\\&\nabla V_{j.}&&=\quad\frac{\partial f_{uj}A_{u}}{\partial V_{j.}}=e_{uj}U_{u}\cdot+\alpha_{\nu}V_{j.},j\in\mathcal{A}_{u}\\&\nabla b_{l}&&=\quad\frac{\partial f_{uj}A_{u}}{\partial B_{l}}=\sum_{j\in A_{u}}(-e_{uj})+\beta_{\nu}B_{l}\\&\nabla V_{l.}&&=\quad\frac{\partial V_{uj}A_{u}}{\partial V_{l.}}=\sum_{j\in A_{uj}}(-e_{uj})U_{u}^{-j}+\alpha_{\nu}V_{l.}\\&\nabla W_{l.}&&=\quad\frac{\partial f_{uj}A_{u}}{\partial W_{l.}}=\sum_{j\in A_{u}}(-e_{uj})\frac{V_{l.}}{|T_{u}\setminus\{l\}|^{\alpha}}-\frac{V_{l.}}{|T_{u}|^{\alpha}})+\alpha_{w}W_{l.},i^{\prime}\in\mathcal{I}_{u}\setminus\{l\}\\&\nabla W_{l.}&&=\quad\frac{\partial f_{uj}A_{u}}{\partial W_{l.}}=\sum_{j\in A_{u}}(-e_{uj})\frac{-V_{l.}}{|T_{uj}|^{\alpha}}+\alpha_{w}W_{l.}\end{aligned}$$

where $e_{uij}=(1-(\hat{r}_{ui}-\hat{r}_{uj}))/|\mathcal{A}_u|.$

Note that we do NOT have to save $V_i.$ and $V_j.$ before they are updated, though we should NOT use the recently updated
parameters but strictly follow the update rules. Please see the details in Algorithm 1, where we use $x$ and $x_{2}$ to save the $V_{i.}$ and
$V_i.$ before they are updated.

For a triple $(u,i,\mathcal{A}_u)$, we have the gradients,

$$\begin{aligned}b_{j}&=\quad b_j-\gamma\nabla b_j\\V_{j.}&=\quad V_{j.}-\gamma\nabla V_j.\\b_{i}&=\quad b_i-\gamma\nabla b_i\\V_{i.}&=\quad V_{i.}-\gamma\nabla V_{i.}\\W_{i^{\prime}.}&=\quad W_{i^{\prime}.}-\gamma\nabla W_{i^{\prime}.},i^{\prime}\in\mathcal{I}_u\backslash\{i\}\\W_{i.}&=\quad W_{i.}-\gamma\nabla W_{i.}\end{aligned}$$

where $\gamma$ is the learning rate.

- user number: n = 943; item number: m = 1682

- We use the statistics of training data to initialize the model parameters,

$$\begin{aligned}b_{i}&=\sum_{u=1}^ny_{ui}/n-\mu\\b_{j}&=\sum_{u=1}^ny_{uj}/n-\mu\\V_{ik}&=\quad(r-0.5)\times0.01,k=1,\ldots,d\\W_{i^{\prime}k}&=\quad(r-0.5)\times0.01,k=1,\ldots,d\end{aligned}$$


where r (0 ≤ r < 1) is a random variable, and $\mu=\sum_u=1^n\sum_{i=1}^my_{ui}/n/m.$

- \begin{aligned}&\text{We fin }\alpha=0.5,\gamma=0.01\mathrm{~and~}|A_{u}|=3\text{, and search the best values of}\\&\text{the following parameters,}\\&\mathrm{~Ф~}\alpha_{v}=\alpha_{w}=\beta_{v}\in\{0.001,0.01,0.1\}\\&\mathrm{~Ф~}T\in\{100,500,1000\}\\&\mathrm{Finally,we~use~}\alpha=0.5,\gamma=0.01,|\mathcal{A}_{u}|=3,\alpha_{v}=\alpha_{w}=\beta_{v}=0.01\mathrm{~and}\\&T=1000,\text{which performs best in our experiments.}\end{aligned}



In [ ]:
import math
import random
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

class FISM_auc:
    def __init__(self, train_data_file, test_data_file, T=1000, d=64, learning_rate=0.01, alpha=0.5,
                 p=3, alpha_v=0.01, alpha_w=0.01, beta_v=0.01):
        self.p = p
        self.T = T
        self.d = d
        self.alpha = alpha
        self.learning_rate = learning_rate
        self.alpha_v = alpha_v
        self.alpha_w = alpha_w
        self.beta_v = beta_v
        
        self.user_num = 943
        self.item_num = 1682
        self.items = set(range(1, self.item_num + 1))
        
        # Initialize parameters
        self.bi = np.zeros(self.item_num + 1)
        self.V = (np.random.rand(self.item_num + 1, self.d) - 0.5) * 0.01
        self.W = (np.random.rand(self.item_num + 1, self.d) - 0.5) * 0.01
        
        # Load data
        self.train_user_items, miu = self._load_data(train_data_file)
        self.test_user_items = self._load_test_data(test_data_file)
        
        # Initialize item biases
        for i in range(1, self.item_num + 1):
            self.bi[i] = len(self.train_user_items.get(i, set())) / self.user_num - miu

    def _load_data(self, filename):
        data = pd.read_csv(filename, sep='\t', header=None, 
                          names=['user', 'item', 'rating', 'timestamp'])
        data = data[data['rating'] > 3]
        
        user_items = {}
        item_users = {}
        total = 0
        for _, row in data.iterrows():
            user = row['user']
            item = row['item']
            user_items.setdefault(user, set()).add(item)
            item_users.setdefault(item, set()).add(user)
            total += 1
        
        miu = total / (self.user_num * self.item_num)
        return user_items, miu

    def _load_test_data(self, filename):
        data = pd.read_csv(filename, sep='\t', header=None,
                          names=['user', 'item', 'rating', 'timestamp'])
        data = data[data['rating'] > 3]
        
        test_user_items = {}
        for _, row in data.iterrows():
            user = row['user']
            item = row['item']
            test_user_items.setdefault(user, set()).add(item)
        return test_user_items

    def _sample_negatives(self, user, num):
        pos_items = self.train_user_items.get(user, set())
        candidates = list(self.items - pos_items)
        return random.sample(candidates, num) if len(candidates) >= num else []

    def train(self):
        for epoch in range(self.T):
            total_loss = 0.0
            for user in tqdm(self.train_user_items, desc=f'Epoch {epoch+1}'):
                pos_items = self.train_user_items[user]
                for i in pos_items:
                    A_u = self._sample_negatives(user, self.p)
                    if not A_u:
                        continue
                    
                    # Calculate U_u^{-i}
                    items_minus_i = [item for item in pos_items if item != i]
                    len_minus_i = len(items_minus_i)
                    sum_W_minus_i = np.sum(self.W[items_minus_i], axis=0) if items_minus_i else 0
                    U_u_minus_i = sum_W_minus_i / (len_minus_i ** self.alpha) if len_minus_i > 0 else 0
                    r_ui = self.bi[i] + np.dot(U_u_minus_i, self.V[i])
                    
                    # Calculate U_u
                    len_total = len(pos_items)
                    sum_W_total = np.sum(self.W[list(pos_items)], axis=0)
                    U_u = sum_W_total / (len_total ** self.alpha) if len_total > 0 else 0
                    
                    sum_e = 0.0
                    sum_grad_Vi = np.zeros(self.d)
                    
                    for j in A_u:
                        r_uj = self.bi[j] + np.dot(U_u, self.V[j])
                        e = (1 - (r_ui - r_uj)) / len(A_u)
                        
                        # Update j parameters
                        self.bi[j] -= self.learning_rate * (e + self.beta_v * self.bi[j])
                        grad_Vj = e * U_u + self.alpha_v * self.V[j]
                        self.V[j] -= self.learning_rate * grad_Vj
                        
                        sum_e += e
                        sum_grad_Vi += (-e) * U_u_minus_i
                        
                        # Update W parameters
                        term_i_prime = (-e) * (self.V[j] / (len_minus_i ** self.alpha) - 
                                             self.V[j] / (len_total ** self.alpha))
                        if items_minus_i:
                            self.W[items_minus_i] -= self.learning_rate * (term_i_prime + 
                                                                          self.alpha_w * self.W[items_minus_i])
                        term_i = (-e) * (-self.V[j] / (len_total ** self.alpha))
                        self.W[i] -= self.learning_rate * (term_i + self.alpha_w * self.W[i])
                    
                    # Update i parameters
                    self.bi[i] -= self.learning_rate * ((-sum_e) + self.beta_v * self.bi[i])
                    grad_Vi = sum_grad_Vi + self.alpha_v * self.V[i]
                    self.V[i] -= self.learning_rate * grad_Vi
                    
                    total_loss += (1 - (r_ui - np.mean([r_uj for j in A_u]))**2
            
            print(f"Average Loss: {total_loss/len(self.train_user_items):.4f}")

    def test(self, K=5):
        hits = 0
        total_rec = 0
        total_relevant = 0
        
        for user in self.test_user_items:
            pos_items = self.test_user_items[user]
            if not pos_items:
                continue
                
            # Calculate U_u
            train_items = self.train_user_items.get(user, set())
            len_total = len(train_items)
            if len_total == 0:
                continue
                
            sum_W = np.sum(self.W[list(train_items)], axis=0)
            U_u = sum_W / (len_total ** self.alpha)
            
            # Generate scores
            scores = []
            for item in self.items - train_items:
                score = self.bi[item] + np.dot(U_u, self.V[item])
                scores.append( (item, score) )
            
            # Get top-K
            scores.sort(key=lambda x: -x[1])
            top_items = [item for item, _ in scores[:K]]
            
            # Calculate metrics
            hits += len(set(top_items) & pos_items)
            total_rec += K
            total_relevant += len(pos_items)
        
        precision = hits / total_rec if total_rec > 0 else 0
        recall = hits / total_relevant if total_relevant > 0 else 0
        print(f"Precision@{K}: {precision:.4f}")
        print(f"Recall@{K}: {recall:.4f}")

if __name__ == "__main__":
    start = time.time()
    model = FISM_auc(r"E:\datasets\ml-100k\u1.base", r"E:\datasets\ml-100k\u1.test")
    model.train()
    model.test()
    print(f"Total time: {time.time()-start:.2f}s")